In [ ]:
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
start_time = dt.datetime.now()

In [ ]:
data = pd.read_csv('/content/sample_data/card transactions.csv')

In [ ]:
#drop blank columns
data.dropna(how = 'all', axis = 1, inplace = True)
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96753 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recnum             96753 non-null  int64         
 1   Cardnum            96753 non-null  int64         
 2   Date               96753 non-null  datetime64[ns]
 3   Merchnum           93378 non-null  object        
 4   Merch description  96753 non-null  object        
 5   Merch state        95558 non-null  object        
 6   Merch zip          92097 non-null  float64       
 7   Transtype          96753 non-null  object        
 8   Amount             96753 non-null  float64       
 9   Fraud              96753 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 7.4+ MB


In [ ]:
data.head(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
5,6,5142149874,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0
6,7,5142189277,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0
9,10,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [ ]:
data['Merchnum'] = data['Merchnum'].replace({'0':np.nan})

In [ ]:
data['Merchnum'].isnull().sum()

3606

In [ ]:
merchdes_merchnum = {}

for index, merchdes in data[data['Merch description'].notnull()][data['Merchnum'].notnull()]['Merch description'].items():
  if pd.isnull(merchdes) == True:
    continue
  elif merchdes not in merchdes_merchnum:
    merchdes_merchnum[merchdes] = data.loc[index, 'Merchnum']

In [ ]:
#fill in by mapping with Merch description
data['Merchnum'] = data['Merchnum'].fillna(data['Merch description'].map(merchdes_merchnum))

In [ ]:
data['Merchnum'].isnull().sum()

2449

In [ ]:
#assign unknown for adjustments transactions
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [ ]:
data['Merchnum'].isnull().sum()

1758

In [ ]:
data.loc[data.Merchnum.isna(), 'Merch description'].unique()[:20]

array(['CONVENIENCE CHECK FEE(%)', 'MONTGOMERY COLLEGE-PHONE',
       'CHARLESTON CO PARKING OP0', 'PACKAGE PLACE  THE',
       'CUBIX CORPORATION', 'SIGNAL GRAPHICS PRINTING',
       'C & M OFFICE EQUIPMENT', "TOMMY'S TRAILERS",
       'Z WORLD/RABBIT SEMICONDUC', 'IMPAC/TRI-COUNTY/FREED',
       'REPROGRPHC TECHNLGIES INC', 'STP SPECIALITY TECH',
       'VANGARD INTERNAITONAL', 'BLACKWELL SCIENCE', 'CDN ISOTOPES INC',
       'INTERACTIVE SOFTWARE S', 'H R WILLIAMS MILL SUPP',
       'ELSEVIER SCIENCE BV', 'COLORADO GARDEN SHOW',
       'PEARSON EDUCATION CANADA'], dtype=object)

In [ ]:
data.loc[data.Merchnum.isna(), 'Merch description'].nunique()
#1758 NULL Merchnums with 667 unique descriptions

667

In [ ]:
#adding new merchnum
#each new unique merchnum will be max(merchnum) +1
merchnum_create = {}
max_merchnum = pd.to_numeric(data.Merchnum, errors = 'coerce').max()
for merch_desc in data.loc[data.Merchnum.isna(), 'Merch description'].unique():
  merchnum_create[merch_desc] = str(int(max_merchnum +1))
  max_merchnum +=1

In [ ]:
#fill in by mapping with Merch description (newly created merchnum)
data['Merchnum'] = data['Merchnum'].fillna(data['Merch description'].map(merchnum_create))

In [ ]:
for i in data.columns:
  print(i, data[i].isnull().sum())

Recnum 0
Cardnum 0
Date 0
Merchnum 0
Merch description 0
Merch state 1195
Merch zip 4656
Transtype 0
Amount 0
Fraud 0


In [ ]:
data['Merch state'].isnull().sum()

1195

In [ ]:
data[(data['Merch state'].isnull()) & (data['Merch zip'].notnull())]['Merch zip'].unique()

array([9.2600e+02, 9.2900e+02, 1.4000e+03, 6.5132e+04, 8.6899e+04,
       2.3080e+04, 6.0528e+04, 9.3400e+02, 9.0200e+02, 7.3800e+02,
       9.0805e+04, 7.6302e+04, 9.0000e+00, 9.1400e+02, 6.0000e+00,
       9.5461e+04, 5.0823e+04, 2.0000e+00, 4.8700e+04, 6.8000e+02,
       1.0000e+00, 6.8100e+02, 6.2300e+02, 7.2600e+02, 9.3600e+02,
       1.2108e+04, 7.9100e+02, 9.0700e+02, 9.2200e+02, 9.2000e+02,
       3.0000e+00, 8.0100e+02, 8.0000e+00, 3.1040e+04, 3.8117e+04,
       4.1160e+04])

In [ ]:
#dict for mapping
zip_state = {}
for index, zip5 in data[data['Merch zip'].notnull()]['Merch zip'].items():
  if zip5 not in zip_state:
    zip_state[zip5]= data.loc[index, 'Merch state']

zip_state['00926'] = 'PR'
zip_state['00929'] = 'PR'
zip_state['00934'] = 'PR'
zip_state['00902'] = 'PR'
zip_state['00738'] = 'PR'
zip_state['90805'] = 'CA'
zip_state['76302'] = 'TX'
zip_state['00914'] = 'PR'
zip_state['95461'] = 'CA'
zip_state['00680'] = 'PR'
zip_state['00623'] = 'PR'
zip_state['00726'] = 'PR'
zip_state['00936'] = 'PR'
zip_state['12108'] = 'NY'
zip_state['00791'] = 'PR'
zip_state['00907'] = 'PR'
zip_state['00922'] = 'PR'
zip_state['00920'] = 'PR'
zip_state['00801'] = 'VI'
zip_state['31040'] = 'GA'
zip_state['41160'] = 'KY'
zip_state['00681'] = 'PR'

In [ ]:
merchnum_state = {}
for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
  if merchnum not in merchnum_state:
    merchnum_state [merchnum] = data.loc[index, 'Merch state']

In [ ]:
merchdes_state = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
  if merchdes not in merchdes_state:
    merchdes_state [merchdes] = data.loc[index, 'Merch state']

In [ ]:
#fill in by mapping with zip, merchnum and merch description
data['Merch state'] = data['Merch state'].fillna(data['Merch zip'].map(zip_state))
data['Merch state'] = data['Merch state'].fillna(data['Merchnum'].map(merchnum_state))
data['Merch state'] = data['Merch state'].fillna(data['Merch description'].map(merchdes_state))

In [ ]:
#assign unknown for adjustments transactions
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [ ]:
data['Merch state'].isnull().sum()

521

In [ ]:
# change non-US states
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'PR', np.nan, 'unknown']

In [ ]:
for index, state in data['Merch state'].items():
  if state not in states:
    data.loc[index, 'Merch state'] = 'foreign'

In [ ]:
data['Merch state'].fillna('unknown', inplace = True)
data['Merch state'].isnull().sum()

0

In [ ]:
#CLEAN AND IMPUTE ZIP
data['Merch zip'].isnull().sum()

4656

In [ ]:
merchnum_zip = {}
for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
  if merchnum not in merchnum_zip:
    merchnum_zip [merchnum] = data.loc[index, 'Merch zip']

In [ ]:
merchdes_zip = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
  if merchdes not in merchdes_zip:
    merchdes_zip [merchdes] = data.loc[index, 'Merch zip']

In [ ]:
data['Merch zip'] = data['Merch zip'].fillna(data['Merchnum'].map(merchnum_zip))
data['Merch zip'] = data['Merch zip'].fillna(data['Merch description'].map(merchdes_zip))

In [ ]:
data['Merch zip'].isnull().sum()

3014

In [ ]:

data['Merch zip'] = data['Merch zip'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch zip'] = data['Merch zip'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [ ]:
data['Merch zip'].isnull().sum()

2332

In [ ]:
data['Merch zip'].fillna('unknown', inplace = True)
data['Merch zip'].isnull().sum()

0

In [ ]:
data['Transtype'].isnull().sum()

0

In [ ]:
data['Fraud'].isnull().sum()

0

In [ ]:
data.shape

(96753, 10)

In [ ]:
#remove outliers 
max_index = data['Amount'].idxmax()
data = data.drop(max_index)

In [ ]:
data.shape

(96752, 10)

In [ ]:
#all but the 'P' transactions 
data = data[data['Transtype'] == 'P']

In [ ]:
data.shape

(96397, 10)

In [ ]:
data.to_csv('card transactions after cleaning.csv', index=False)